In [1]:
from mlc_dac.streaming import CachedPadding1d
from tvm import relax, topi
from tvm.script import ir as I
from tvm.script import relax as R
from tvm.script import tir as T
from tvm.relax.frontend import nn
import tvm
import numpy as np


class StreamingConv(nn.Module):
    def __init__(self):
        self.cache = CachedPadding1d(3, False, 1)

    def forward(self, x: nn.Tensor):
        y = self.cache(x)
        return y, self.cache.view(1)
    
mod, params = StreamingConv().export_tvm(
    {
        "forward": {
            "x": nn.spec.Tensor([1, 1, 10], "float32"),
        },
    }
)

device = tvm.device("metal")
target = tvm.target.Target.from_device(device)

with target:
    mod = tvm.ir.transform.Sequential(
        [
            relax.get_pipeline("zero"),
        ]
    )(mod)

mod.show()

ex = relax.build(mod, "llvm")
vm = relax.VirtualMachine(ex, tvm.cpu())
effects = vm["_initialize_effect"]()

x = tvm.nd.array(np.random.rand(1, 1, 10).astype("float32"))
print(x.asnumpy())
o1, o2 = vm["forward"](x, *effects)[0]
print(o1.asnumpy())
print(o2.asnumpy())

x = tvm.nd.array(np.random.rand(1, 1, 10).astype("float32"))
print(x.asnumpy())
o1, o2 = vm["forward"](x, *effects)[0]
print(o1.asnumpy())
print(o2.asnumpy())


[23:09:16] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[23:09:16] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[23:09:16] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`


[[[0.545322   0.26385272 0.30202466 0.08276599 0.20769066 0.913202
   0.85411614 0.28952256 0.02826601 0.28631422]]]
[[[0.         0.         0.         0.545322   0.26385272 0.30202466
   0.08276599 0.20769066 0.913202   0.85411614 0.28952256 0.02826601
   0.28631422]]]
[[[0.28952256 0.02826601 0.28631422]]]
[[[0.47812256 0.21356206 0.47955388 0.01061336 0.54981154 0.21186082
   0.10509059 0.41574165 0.08226193 0.26996234]]]
[[[0.28952256 0.02826601 0.28631422 0.47812256 0.21356206 0.47955388
   0.01061336 0.54981154 0.21186082 0.10509059 0.41574165 0.08226193
   0.26996234]]]
[[[0.41574165 0.08226193 0.26996234]]]


In [2]:
import numpy as np
import tvm
from tvm import relax
from tvm.relax.frontend import nn
from tvm.target import Target
from tvm import dlight as dl


from mlc_dac.layers import CachedConv1d

cached_conv = CachedConv1d(5, 10, 3, padding=1)

mod_cached, params_cached = cached_conv.export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 5, 1000), "float32")}}, debug=True
)

mod, params = nn.Conv1D(5, 10, 3, padding=1).export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 5, 1000), "float32")}}, debug=True
)

data = np.random.rand(1, 5, 1000).astype("float32")
weight = np.random.rand(10, 5, 3).astype("float32")
bias = np.random.rand(10).astype("float32")

seq = tvm.transform.Sequential(
    [
        tvm.relax.transform.LegalizeOps(),
        dl.ApplyDefaultSchedule(
            dl.gpu.Fallback(),
        ),
    ]
)

device = tvm.metal()
target = Target.from_device(device)
with target:
    mod_cached = seq(mod_cached)
ex = relax.build(mod_cached, target)
device = tvm.metal()

vm = relax.VirtualMachine(ex, device)
tvm_data = tvm.nd.array(data, device=device)
params = [weight, bias]
params = [tvm.nd.array(param, device=device) for param in params]

effects = vm["_initialize_effect"]()
output_tvm_cached = vm["forward"](tvm_data, *effects, *params)[0]

seq = tvm.transform.Sequential(
    [
        tvm.relax.transform.LegalizeOps(),
        dl.ApplyDefaultSchedule(
            dl.gpu.Fallback(),
        ),
    ]
)

device = tvm.metal()
target = Target.from_device(device)
with target:
    mod = seq(mod)
ex = relax.build(mod, target)
device = tvm.metal()

vm = relax.VirtualMachine(ex, device)
tvm_data = tvm.nd.array(data, device=device)
params = [weight, bias]
params = [tvm.nd.array(param, device=device) for param in params]

effects = vm["_initialize_effect"]()
output_tvm = vm["forward"](tvm_data, *effects, *params)[0]

print(output_tvm_cached.asnumpy().shape)
print(output_tvm.asnumpy().shape)
print(
    np.allclose(
        output_tvm.asnumpy()[..., : -cached_conv.cumulative_delay],
        output_tvm_cached.asnumpy()[..., cached_conv.cumulative_delay :],
    )
)

(1, 10, 1000)
(1, 10, 1000)
True


In [3]:
import numpy as np
import tvm
from tvm import relax
from tvm.relax.frontend import nn
from tvm.target import Target
from tvm import dlight as dl

from mlc_dac.layers import CachedConvTranspose1d

cached_conv = CachedConvTranspose1d(5, 10, 3, padding=1)

mod_cached, params_cached = cached_conv.export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 5, 1000), "float32")}}, debug=True
)

mod, params = nn.ConvTranspose1D(5, 10, 3, padding=1).export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 5, 1000), "float32")}}, debug=True
)

data = np.random.rand(1, 5, 1000).astype("float32")
weight = np.random.rand(5, 10, 3).astype("float32")
bias = np.random.rand(10).astype("float32")

seq = tvm.transform.Sequential(
    [
        tvm.relax.transform.LegalizeOps(),
        dl.ApplyDefaultSchedule(
            dl.gpu.Fallback(),
        ),
    ]
)

device = tvm.metal()
target = Target.from_device(device)
with target:
    mod_cached = seq(mod_cached)
ex = relax.build(mod_cached, target)
device = tvm.metal()

vm = relax.VirtualMachine(ex, device)
tvm_data = tvm.nd.array(data, device=device)
params = [weight, bias]
params = [tvm.nd.array(param, device=device) for param in params]

effects = vm["_initialize_effect"]()
output_tvm_cached = vm["forward"](tvm_data, *effects, *params)[0]

seq = tvm.transform.Sequential(
    [
        tvm.relax.transform.LegalizeOps(),
        dl.ApplyDefaultSchedule(
            dl.gpu.Fallback(),
        ),
    ]
)

device = tvm.metal()
target = Target.from_device(device)
with target:
    mod = seq(mod)
ex = relax.build(mod, target)
device = tvm.metal()

vm = relax.VirtualMachine(ex, device)
tvm_data = tvm.nd.array(data, device=device)
params = [weight, bias]
params = [tvm.nd.array(param, device=device) for param in params]

effects = vm["_initialize_effect"]()
output_tvm = vm["forward"](tvm_data, *effects, *params)[0]

print(output_tvm_cached.asnumpy().shape)
print(output_tvm.asnumpy().shape)

print(
    np.allclose(
        output_tvm.asnumpy()[..., : -cached_conv.cumulative_delay],
        output_tvm_cached.asnumpy()[..., cached_conv.cumulative_delay :],
    )
)

(1, 10, 1000)
(1, 10, 1000)
True


In [4]:
import numpy as np
import tvm
from tvm import relax
from tvm.relax.frontend import nn
from tvm.target import Target
from tvm import dlight as dl

from mlc_dac.layers import CachedWNConv1d, WNConv1d

cached_conv = CachedWNConv1d(5, 10, 10, padding=1)

mod_cached, params_cached = cached_conv.export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 5, 1000), "float32")}}, debug=True
)

mod, params = WNConv1d(5, 10, 10, padding=1).export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 5, 1000), "float32")}}, debug=True
)

data = np.random.rand(1, 5, 1000).astype("float32")
weight_g_data = np.random.rand(10, 1, 1).astype("float32")
weight_v_data = np.random.rand(10, 5, 10).astype("float32")
bias_data = np.random.rand(10).astype("float32")

target = Target.from_device("metal")
with target:
    mod_cached = relax.transform.LegalizeOps()(mod_cached)
    mod_cached = dl.ApplyDefaultSchedule(
        dl.gpu.Fallback(),
    )(mod_cached)
ex = relax.build(mod_cached, target)
device = tvm.metal()

vm = relax.VirtualMachine(ex, device)
tvm_data = tvm.nd.array(data, device=device)
params = [weight_g_data, weight_v_data, bias_data]
params = [tvm.nd.array(param, device=device) for param in params]

effects = vm["_initialize_effect"]()
out_cached = vm["forward"](tvm_data, *effects, *params)[0].numpy()

target = Target.from_device("metal")
with target:
    mod = relax.transform.LegalizeOps()(mod)
    mod = dl.ApplyDefaultSchedule(
        dl.gpu.Fallback(),
    )(mod)
ex = relax.build(mod, target)
device = tvm.metal()

vm = relax.VirtualMachine(ex, device)
tvm_data = tvm.nd.array(data, device=device)
params = [weight_g_data, weight_v_data, bias_data]
params = [tvm.nd.array(param, device=device) for param in params]

effects = vm["_initialize_effect"]()
out = vm["forward"](tvm_data, *effects, *params)[0].numpy()

print(out_cached.shape)
print(out.shape)
print(np.allclose(out_cached[..., cached_conv.cumulative_delay :], out[..., : -cached_conv.cumulative_delay]))

(1, 10, 993)
(1, 10, 993)
True


In [5]:
import numpy as np
import tvm
from tvm import relax
from tvm.relax.frontend import nn
from tvm.target import Target
from tvm import dlight as dl

from mlc_dac.layers import CachedWNConvTranspose1d, WNConvTranspose1d

cached_conv = CachedWNConvTranspose1d(5, 10, 10, padding=1)

mod_cached, params_cached = cached_conv.export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 5, 1000), "float32")}}, debug=True
)

mod, params = WNConvTranspose1d(5, 10, 10, padding=1).export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 5, 1000), "float32")}}, debug=True
)

data = np.random.rand(1, 5, 1000).astype("float32")
weight_g_data = np.random.rand(5, 1, 1).astype("float32")
weight_v_data = np.random.rand(5, 10, 10).astype("float32")
bias_data = np.random.rand(10).astype("float32")

target = Target.from_device("metal")
with target:
    mod_cached = relax.transform.LegalizeOps()(mod_cached)
    mod_cached = dl.ApplyDefaultSchedule(
        dl.gpu.Fallback(),
    )(mod_cached)
ex = relax.build(mod_cached, target)
device = tvm.metal()

vm = relax.VirtualMachine(ex, device)
tvm_data = tvm.nd.array(data, device=device)
params = [weight_g_data, weight_v_data, bias_data]
params = [tvm.nd.array(param, device=device) for param in params]

effects = vm["_initialize_effect"]()
out_cached = vm["forward"](tvm_data, *effects, *params)[0].numpy()

target = Target.from_device("metal")
with target:
    mod = relax.transform.LegalizeOps()(mod)
    mod = dl.ApplyDefaultSchedule(
        dl.gpu.Fallback(),
    )(mod)
ex = relax.build(mod, target)
device = tvm.metal()

vm = relax.VirtualMachine(ex, device)
tvm_data = tvm.nd.array(data, device=device)
params = [weight_g_data, weight_v_data, bias_data]
params = [tvm.nd.array(param, device=device) for param in params]

effects = vm["_initialize_effect"]()
out = vm["forward"](tvm_data, *effects, *params)[0].numpy()

print(out_cached.shape)
print(out.shape)
print(np.allclose(out_cached[..., cached_conv.cumulative_delay :], out[..., : -cached_conv.cumulative_delay]))

(1, 10, 1007)
(1, 10, 1007)
True


In [6]:
import numpy as np
import tvm
from tvm import relax
from tvm.relax.frontend import nn
from tvm.target import Target
from tvm import dlight as dl

from mlc_dac.layers import AlignBranches, CachedConv1d, Identity

conv = CachedConv1d(1, 1, 5, padding=2)
model = AlignBranches(
    conv,
    Identity(),
    delays=[conv.cumulative_delay, 0],
)

mod, params = model.export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 1, 10), "float32")}}, debug=True
)
mod.show()

In [1]:
from mlc_dac.dac import DAC
import numpy as np
import tvm
from tvm import relax
from tvm.relax.frontend import nn
from tvm.target import Target
from tvm import dlight as dl

mod_from_relax, params_from_relax = DAC().export_tvm(
    {"encode": {"audio_data": nn.spec.Tensor((32, 1, 100000), "float32")}}, debug=True
)

seq = tvm.transform.Sequential(
    [
        tvm.relax.transform.LegalizeOps(),
        tvm.relax.transform.AnnotateTIROpPattern(),
        tvm.relax.transform.FoldConstant(),
        tvm.relax.transform.FuseOps(),
        tvm.relax.transform.FuseTIR(),
        dl.ApplyDefaultSchedule(
            # dl.gpu.Matmul(),
            dl.gpu.GEMV(),
            dl.gpu.Reduction(),
            dl.gpu.GeneralReduction(),
            dl.gpu.Fallback(),
        ),
    ]
)
target = Target.from_device("metal")
with target:
    mod_from_relax = seq(mod_from_relax)

[23:37:59] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[23:37:59] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[23:37:59] /Users/cfruan/Documents/tvm-unity/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.1 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
